In [ ]:
from langchain.agents import load_tools  # This will allow us to load tools we need
from langchain.agents import initialize_agent


from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
#from langchain.llms import OpenAI

## We can also use a model from HuggingFaceHub if we wish to go open-source!

cachee_dir="/Users/giannitallarita/Downloads/titanic"

model_id = "EleutherAI/gpt-j-6b"
tokenizer = AutoTokenizer.from_pretrained(model_id , max_length=512, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline(
    "text-generation", model=model,tokenizer=tokenizer, max_new_tokens=512, device_map='auto'
)
jekyll_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



csvd=CSVLoader(file_path='/Users/giannitallarita/Downloads/titanic/train.csv', csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['PassengerId', 'Survived', 'Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
})

In [ ]:
#chunk_size = 500
#chunk_overlap = 0

data = csvd.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

#args = parse_arguments()

    # activate/deactivate the streaming StdOut callback for LLMs
#loader = TextLoader("../../state_of_the_union.txt")
#documents = loader.load()
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#texts = text_splitter.split_documents(data)
texts=data
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(texts, embeddings)
#retriever = db.as_retriever(search_kwargs={"k": target_source_chunks})

qa = RetrievalQA.from_chain_type(llm=jekyll_llm, chain_type="stuff", retriever=db.as_retriever())

In [ ]:
query = "How many columns are there?"
qa.run(query)